In [1]:
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch
import cv2
import os
from time import sleep

c:\Users\texas\anaconda3\envs\MovieDiff\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\users\texas\desktop\personal\aim24s\bryant\mobilesam\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
c:\users\texas\desktop\personal\aim24s\bryant\mobilesam\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_11m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
c:\users\texas\desktop\pers

In [2]:

model_type = "vit_t"
sam_checkpoint = "./MobileSAM/weights/mobile_sam.pt"

device = "cuda" if torch.cuda.is_available() else "cpu"

mobile_sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mobile_sam.to(device=device)
mobile_sam.eval()

Sam(
  (image_encoder): TinyViT(
    (patch_embed): PatchEmbed(
      (seq): Sequential(
        (0): Conv2d_BN(
          (c): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): GELU(approximate='none')
        (2): Conv2d_BN(
          (c): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
    (layers): ModuleList(
      (0): ConvLayer(
        (blocks): ModuleList(
          (0-1): 2 x MBConv(
            (conv1): Conv2d_BN(
              (c): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (act1): GELU(approximate='none')
            (conv2): Conv2d_BN(
 

In [3]:

abspath = os.path.abspath("")
print(os.path.join(os.path.dirname(abspath), "Bryant\\test_imgs\\labrador.jpg"))
image = cv2.imread(os.path.join(os.path.dirname(abspath), "Bryant\\test_imgs\\labrador.jpg"))
# turn into ndarray

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

predictor = SamPredictor(mobile_sam)


c:\Users\texas\Desktop\Personal\AIM24S\Bryant\test_imgs\labrador.jpg


In [8]:
import gradio as gr
import numpy as np

# get stream of images using cv2
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
current_frame = frame
masks = []

def showStream():
    global masks
    global current_frame
    # ind = int(ind)
    # yeild the current frame forever
    while True:
        
        
        # turn bool to int
        # print(len(masks))
        
        if len(masks) == 0:
            print("No masks found")
            yield current_frame, current_frame
            sleep(0.1)
            continue
        
        # if ind >= len(masks):
        #     ind = len(masks) - 1
        
        
        
        # turn (3, 480, 640) (480, 640, 3)
        # masks = np.moveaxis(masks, 0, -1)
        # print(masks.shape, current_frame.shape)
        # print(current_frame)
        # print(masks)
        
        yield current_frame, masks
        sleep(0.01)
    return None

# create a gradio interface and output two images
# iface = gr.Interface(
#     fn=showStream,
#     inputs=None,
#     outputs=["image", "image"],
# ).queue(concurrency_count=5)
# iface.launch()

# create a gradio interface and output two images using blocks
with gr.Blocks() as demo:
    
    with gr.Row():
        out1 = gr.Image(label="Original Image")
        out2 = gr.Image(label="Masked Image")
    # slider = gr.Slider(minimum=0, maximum=10, default=0, label="Slider", step=1)
    btn = gr.Button("Run")
    btn.click(fn=showStream, inputs=None, outputs=[out1, out2])

demo.queue()
demo.launch()

stop_threads = True
sleep(1)
stop_threads = False

def updateMasks():
    global masks
    global current_frame
    while True:
        global stop_threads
        if stop_threads:
            break
        predictor.set_image(current_frame)
        # predictor.set_image(image)
        tmasks, _, _ = predictor.predict()
        tmasks = tmasks[0].astype(int)
        # turn 0 and 1 to 0 and 255
        tmasks = tmasks * 255
        # turn into 3 channels
        masks = np.stack([tmasks] * 3, axis=-1)

# create a thread to update the masks
import threading
t = threading.Thread(target=updateMasks)
t.start()

while True:
    ret, frame = cap.read()
    current_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    if not ret:
        print("Error: failed to capture image")
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


KeyboardInterrupt: 

In [7]:
#get all threads
import threading
stop_threads = True
for thread in threading.enumerate():
    print(thread.name)
    # if the thread contains the word "updateMasks"
    if "updateMasks" in str(thread.name):
        # join the thread
        print("Joining thread")
        # thread.join()
        # print("Thread joined")
        # break
    

MainThread
IOPub
Heartbeat
Control
IPythonHistorySavingThread
Thread-4
Thread-8 (run)
AnyIO worker thread
Thread-13 (run)
AnyIO worker thread


In [ ]:
# print gradio version
print(gr.__version__)
cap.release()
cv2.destroyAllWindows()

3.50.2


3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
